# Init

In [1]:
if 0:
    import sys
    print("Python version")
    print(sys.version)
    print("Version info.")
    print(sys.version_info)

%load_ext autoreload
%autoreload 2

####### INIT ######################################################################################

import datetime
today = datetime.date.today()

from context import rmlib
import rmlib.rmtools as rm
from config import robotConfig

robot = rmlib.robot.Robot( robotConfig )

#Imports 
from time import sleep
import numpy as np
import rmlib
from rmlib.rmtools.assembly_trees import augment_RM

augment_RM( robot )

%store -r

####### LIBS ######################################################################################

# Import Pytrees
import py_trees
from py_trees.tests import Timeout_Success
from py_trees.composites import Sequence , Selector

from assembly_trees import *

import math
from pmath import translate_pose , get_disance_between_poses , pose_components

from utils import is_matx_list
from math import radians

print( "\n##!## READY :", today, "##!##" )

### Timing Test ###
if 0:
    
    import time

    N    = 250
    data = [ None for i in range(N) ]

    print( robot.ft.bias_wrist_force() )
    sleep(0.1)

    bgn = time.time()

    # Take `N` datapoints AQAP
    for i in range(N):
        data[i] = robot.ft.get_wrist_force()

    elapsed = time.time() - bgn

    print( elapsed , "elapsed, Average fetch:" , elapsed/N )
    
    for datum in data[:20]:
        print( datum )

## Bias Test ##
if 0:
    for i in range(20):
        print('##', i+1, '##')
        before = robot.ft.get_wrist_force()
        bgn    = time.time()
        robot.ft.bias_wrist_force()
        print( time.time() - bgn , "[s] bias time" )
        after  = robot.ft.get_wrist_force()
        print( after )
        print( "Diff:" , np.linalg.norm( np.subtract( after, before ) ) , '\n\n' )

Loaded: /home/nvidia/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/nvidia/dev_rmstudio/rmlib Exists?: True
Setting up robot please wait...
my_components: {'hand_config': {'finger_width_inner': 0.0, 'finger_width_outer': 0.015, 'module_name': 'hands.smarthand', 'class_name': 'SmartHand', 'finger_length': 0.0415, 'finger_depth': 0.014}, 'arm_config': {'default_joint_speed': 0.7, 'default_linear_speed': 0.1, 'max_linear_speed': 0.25, 'default_joint_accel': 0.8, 'max_joint_accel': 1.4, 'xmlrpc_port': '8003', 'class_name': 'UR5', 'max_linear_accel': 1.2, 'module_name': 'arms.ur5', 'max_joint_speed': 1.05, 'default_linear_accel': 0.8, 'ip_address': '192.168.0.101'}, 'cam_config': {'module_name': 'cameras.realsense', 'ci_cam_offset': [-0.036, -0.028, -0.079], 'camera_model': 'd410', 'pc_cam_offset': [-0.037, -0.033, -0.079], 'class_name': 'RealSense'}, 'ft_config': {'dataHz': 50.0, 'local_port': 20000, 'remote_port': 10000, 'local_ip': '127.0.0.1', 'filter': {'alpha': 0.012, 'b

# Diagnostic Tests

In [2]:
from utils import HeartRate
import time

# 2021-06-17: These tests are nearly identical. If you are going to write more, then DRY


def test_func_frequency( testFunc, cols = 6, critFunc = None, N = 100, hz = 50 ):
    """ Determine the ability of `testFunc` to maintain the requirement(s) defined by `critFunc` """
    
    def default_criterion( info ):
        """ Return T if observed average frequency is 0.90 of desired or greater, otherwise F """
        return info['avg_freq'] >= info['test']['des_freq']*0.90
    
    # 0. If no criterion is given, assume that we are testing periodicity of `testFunc`
    if critFunc is None:
        critFunc = default_criterion
    # 1. Allocate memory and Set timing vars 
    data = np.zeros( (N,cols) ) # Allocate space for data
    dlay = np.zeros( (N)      ) # Allocate space for delays
    rate = HeartRate( hz ) # ---- Goal is to avoid dipping below this frequency (Starts clock)
    last = time.time() # -------- Start the dealy clock
    # 2. Run the timing test
    for i in range(N):
        data[i] = testFunc.__call__()
        now     = time.time()
        dlay[i] = now - last
        last    = now
        rate.sleep()
    # 3. Organize test results
    results = {
        # Test Params #
        'test': {
            'des_freq': hz, # Desired Frequency
            'N'       : N, #- Number of trials
        },
        # Test Results #
        'avg'     : np.mean( dlay ), # ----- Average delay between runs [s]
        'avg_freq': 1.0 / np.mean( dlay ), # Average frequency [Hz]
        'max'     : np.amax( dlay ), # ----- Max delay between function calls [s]
        'sum'     : sum( dlay ), # --------- Total test duration [s]
        'delay'   : dlay, # ---------------- Delay data
        'data'    : data # ----------------- Function returned data
    }
    # 3. Evaluate and Return test results
    results['pass'] = critFunc( results )
    return results


def test_FT_latency( N = 100, hz = 50 ):
    """ Attempt to test the data latency for the ft sensor, Return PASS/FAIL result """
    return test_func_frequency( 
        robot.ft.get_wrist_force, cols = 6, critFunc = None, 
        N = N, hz = hz 
    )['pass']
    

def test_RTDE_latency( N = 100, hz = 50 ):
    """ Attempt to test the data latency for the ft sensor, Return PASS/FAIL result """
    return test_func_frequency( 
        robot.arm.get_tcp_pose_vec, cols = 6, critFunc = None, 
        N = N, hz = hz 
    )['pass']

# Global Vars

In [3]:
DATA_RATE  = 50
SLEEP_TIME = 10.0

# Tilt Insert: Shaft Recording

In [4]:
from py_trees.decorators import SuccessIsFailure
from py_trees.timers import Timer
from pathlib import Path
import operator
from random import random

statusKey = "insertStatus" # The name of the current logfile will go here
latchAll  = 0


def make_above_pose( trgtPose, vertOffset = 0.070 ):
    """ Return a pose that is vertically offset  """
    rtrnPose = trgtPose.copy()
    rtrnPose[2,3] += vertOffset
    return rtrnPose


def perturb_pose_radially_XY( pose, radMax ):
    """ Return a version of `pose` that is moved in the XY plane up to `radMax` distance """
    rtrnPose = pose.copy()
    dX = dY = 5000.0
    while np.linalg.norm( [dX, dY] ) > radMax:
        dX = -radMax + 2*radMax*random()
        dY = -radMax + 2*radMax*random()
    rtrnPose[0,3] += dX
    rtrnPose[1,3] += dY
    print(  )
    return rtrnPose


class TiltRecoveryLoop_Simple( py_trees.composites.Sequence ):
    """ Tree for retrying Tilt Insertion until it succeeds, No monitoring """
    
    def setup( self ):
        """ Create a log file when the experiment begins """
        self.logPath = self.logPathGen()
        print( "Run log stored in" , self.logPath )
        
    def arrange_logs_and_write( self ):
        """ Gather everything that was logged, arrange in chrono order, and write to CSV """
        events = []
        for log in self.logs:
            events.extend( log.log )
            
         #  7. Define a timestamp comparison for sorting rows of data
        func = operator.itemgetter(0)
        def floatGetter(s):
            return float(func(s))

        #  8. Sort data 
        sortedEvents = sorted( events, key = floatGetter )
        
        with open( self.logPath, 'w', newline='') as csvfile:
            csvWriter = csv.writer( csvfile )
            csvWriter.writerows( sortedEvents )
    
    def __init__( self, Ntries, dwellTime = 10.0, ctrl = None,
                        perturbRad = 0.0 ):
        """ Create a tree """
        
        #### Setup ####
        
        # 0. Main loop should monitor continuously
        super().__init__( name = "Tilt Insert Loop", memory = 0 )
        
        self.zSAFE          = 0.100
        self.plungeDistance = 1.5 * self.zSAFE
        self.setdownForce   = 1.5
        self.descendSpeed   = 0.0225
        self.biasWrist      = 1
        self.Ntries         = Ntries
        self.logs           = list()
        self.logPath        = None
        self.logPathGen     = get_outfile_namer( pathPrefix = "output/" , 
                                                 namePrefix = "SHAFT-TLT-EVENT-LOG",
                                                 filePostfix = "txt" )
        
        
        #### Monitor Subtree ####
        
        recordBehav = Record_Classify( 
            recordFlagKey = "test_FLAG" , dataKey = "test_DATA" , labelKey = statusKey , 
            outFileGenFunc = get_outfile_namer( pathPrefix = "output/" , 
                                                namePrefix = "SHAFT-TLT" ),
            recordHz = DATA_RATE , ctrl = ctrl 
        )
        
        self.add_child( recordBehav )
        
        
        #### Experiment Subtree ####
        
        ### Fetch Subtree ###
        fetchTree = Sequence( memory = 1 )
        
#         print( dir( self ) ) 
        
        pickBgn = EventLogger( msgOrGen = "Pick:Begin", latch = latchAll )
        self.logs.append( pickBgn )
        
        #  1. Go to pickup pose
        goBehav = Jog_Safe( shaftGraspPose , 
                            zSAFE = self.zSAFE ,  
                            hover = 1 ,  
                            ctrl  = ctrl  )

        #  2. Pre-close around the shaft
        closeBehav = Set_Fingers( openState = 0.040 , ctrl = ctrl )   

        #  3. Grasp shaft
        graspBehav = Grasp_at_Pose( shaftGraspPose, 0.015 ,
                                    zApproach=0.050 , zClose=0.0 , wdthNarrow = 0.030 , 
                                    maxIter=5, zFree=0.100 , 
                                    ctrl = ctrl )

        behavPreInsertPose = shaftPreInsertPose.copy()
        
        if perturbRad > 0.0001:
            behavPreInsertPose = perturb_pose_radially_XY( shaftPreInsertPose, perturbRad )
        
        
        #  4. Goto pre-insertion pose
#         jogBehav = Jog_Safe( shaftPreInsertPose , 
        jogBehav = Jog_Safe( behavPreInsertPose , 
                             zSAFE = self.zSAFE ,  
                             hover = 0 ,  
                             ctrl  = ctrl  )
        
        pickEnd = EventLogger( msgOrGen = "Pick:End", latch = latchAll )
        self.logs.append( pickEnd )
        
        fetchTree.add_children( [ pickBgn, goBehav, closeBehav, graspBehav, jogBehav, pickEnd ] )
        
        
        ### Shortcut Subtree ### 
        shortTree = Selector( memory = 1 )
        shortSeq  = Sequence( memory = 1 )
        
        #  5. TODO: SHORTCUT HERE (Monitored Version Only)
        
#         waitBehav = Timer( name="Wait for WRITE", duration=2.0)
        
        #  6. Sibling to the actual insertion subtree kicks off a thread for recording
        
        insertBgn = EventLogger( msgOrGen = "Insert:Begin", latch = latchAll )
        self.logs.append( insertBgn )
        
        #  7. Insertion behavior
#         tiltBehav = Tilt_Insert( shaftPreInsertPose ,
        tiltBehav = Tilt_Insert( behavPreInsertPose ,
                                 tilt_angle_deg = 7.25 , part_offset = 0.016 , dia = 0.015 , 
                                 hackFraction = 0.025, 
                                
                                 ## DANGER: EXPERIMENT CHANGE ##
                                 touch_force = 3.0 , insert_force = 4.0 , max_movement = 0.085 , 
                                
                                 biasWrist = self.biasWrist , offset_dir = np.array( [ 1.0, 0.0, 0.0 ] ), 
                                 posnMargin = 0.003 , orntMargin = 2.0/180 , 
                                 finalInsertHandZ = fatPegFullInsertZ , 
                                 recordFlagKey = "test_FLAG" ,
                                 dropSpeed = 0.0313 , dropAccel = 0.080 , ctrl = ctrl )
        tiltStatus = Store_Subtree_Status( tiltBehav, statusKey )
        
        insertEnd = EventLogger( msgOrGen = "Insert:End", latch = latchAll )
        self.logs.append( insertEnd )
        
        

#         shortSeq.add_children( [waitBehav, insertBgn, tiltStatus, insertEnd] )
        shortSeq.add_children( [insertBgn, tiltStatus, insertEnd] )
        shortTree.add_children( [ shortSeq, insertEnd ] )
        
        ### Reset Subtree ###
        resetTree = Sequence( memory = 1 )
        
        resetBgn = EventLogger( msgOrGen = "Reset:Begin", latch = latchAll )
        self.logs.append( resetBgn )
        
        
        #  8. Goto pickup pose "goBehav"
        
        #  9. Move the shaft gently to the table
        placeBehav = Move_to_Contact( relMove = [ 0 , 0 , -self.plungeDistance ] , 
                                      Fmag = 3.0 , speed = self.descendSpeed , 
                                      accel = 0.08,
                                      biasWrist = self.biasWrist , ctrl = ctrl )
        
        
        #  9. Drop, whether we won or not
        dropBehav = Set_Fingers( openState = 1.0 , ctrl = ctrl )
        
        resetEnd = EventLogger( msgOrGen = "Reset:End", latch = latchAll )
        self.logs.append( resetEnd )
        
        
        # 10. Construct the reset subtree
        resetTree.add_children( [resetBgn, goBehav, placeBehav, dropBehav, resetEnd] )
        
        
        #### Main Retry Loop ####
        
        #  0. Create the main MC driver
        experimentTree = py_trees.composites.Sequence( memory = 1 )
        experimentRoot = py_trees.composites.Selector( memory = 1 )
        
        
        #  1. Add the decorator
        retryLoop = Run_to_X_Failures_DECO(  
            experimentRoot ,
            X_allowedFails = Ntries
        )
        
        ### Success Subtree ###
        successTree = Sequence( memory = 1 )
        successTree.add_children( [goBehav, placeBehav, dropBehav] )
        
        # We can only reach the successtree after winning the retryLoop
        experimentTree.add_children( [retryLoop, successTree] )
        self.add_child( experimentTree )

        # 11. Add the record/insert/reset siblings as children
        experimentRoot.add_children( [
            SuccessIsFailure( fetchTree ), 
#             shortSeq, 
#             SuccessIsFailure( shortSeq ), 
            shortSeq,
            SuccessIsFailure( resetTree ),
        ] )
        
print( TiltRecoveryLoop_Simple , "is ready!" )

<class '__main__.TiltRecoveryLoop_Simple'> is ready!


# Tilt Insert Pipeline

In [ ]:
N        =  8 # Number of total trials
N_retry  =  1 # Number of max tries per trial (including original)
stpSleep =  5 # Seconds to pause after each trial
div      =  5 # Number of trials between breaks
divSleep = 60 # Seconds to pause for each break
f_divBrk =  0 # Flag for whether to take a break based on trail number
chkHosts  = ['192.168.0.102', '192.168.0.100'] # Ping PC+FT to gauge net health
ft_host   = chkHosts[1]
pause_s  =  5

from time import sleep
from http.client import CannotSendRequest
from utils import network_thermal_health, temp_trends

times = []
temps = []
tMtrs = []
dlays = []

for i in range( 1 , N+1 ):
    
    try:
        print( "Iteration" , i , "of" , N )
        health = network_thermal_health( remoteHosts = chkHosts )
        T_mtr  = robot.hand.get_motor_temperature()
        times.append( health['time']                        )
        temps.append( health['avg_temp']                    )
        tMtrs.append( sum(T_mtr)/len(T_mtr)                 )
        dlays.append( health['network'][ft_host]['latency'] )
        
        latency = dlays[-1]
        i_wait  = 0
        
        ##### DIAGNOSTICS #####
        
        while latency > 800.0: #350: #275: #800.0:
            i_wait += 1
            print( "" )
            robot.ft.set_pause( 1 )
            print( "\n\n... EXCESSIVE LATENCY", latency, ": ATTEMPT PAUSE?", 
                   pause_s, "seconds! ... Wait #", i_wait,"\n" )
            sleep( pause_s )
            chkTemp = network_thermal_health( remoteHosts = chkHosts )['avg_temp']
            print( "Temp Change over rest:", chkTemp - temps[-1] )
            robot.ft.set_pause( 0 )
            sleep( 0.5 )
            latency = network_thermal_health( remoteHosts = chkHosts )['network'][ft_host]['latency']
        
        if 0:
            print( "FT   Latency Pass?:", test_FT_latency()   )
            print( "RTDE Latency Pass?:", test_RTDE_latency() )
        
        
        # 6. Sleep for human reset
        if i > 1:
            sleep( stpSleep )
        
        # Only perturb for half of the trials
        if i%2==1:
            nudge = 0.002
        else:
            nudge = 0.000
        print( "\nUsing nudge:", nudge, '\n' )
            
            
        # 1. Reset tree
        rootNode = TiltRecoveryLoop_Simple( Ntries=N_retry, ctrl=robot, perturbRad=nudge )
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode, N = 5000, tickPause = 0.25, 
                           breakOnFailure = 1, breakOnSuccess = 1, Nverb = 100 )
        
        # 3. Release
        robot.hand.release()
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )
            
        print( "About to write logs", end = " ..." )
        rootNode.arrange_logs_and_write()
        print( "Written!" )
        
        # If set breaks enabled, take a `divSleep` seconds break after a set of `div` trials
        if f_divBrk and (i%div==0) and ((i+1)<N):
            robot.ft.set_pause( 1 )
            print( "\n\n... Taking a rest for", divSleep, "seconds! ...\n" )
            sleep( divSleep )
            chkTemp = network_thermal_health( remoteHosts = chkHost )['avg_temp']
            print( "Temp Change over rest:", chkTemp - temps[-1] )
            robot.ft.set_pause( 0 )
            sleep( 0.5 )
            
        if i == N:
            raise KeyboardInterrupt( "Stop these weird loop errors" )

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break
        
    except CannotSendRequest:
        print( "There was a communication error at iteration" , i+1 )
        break
        
    except ConnectionRefusedError:
        print( "There was a connection refused error at iteration" , i+1 , 
               ", Maybe you pressed 'Stop'?" )
        break
        
    
        
temp_trends( times, [temps, tMtrs, dlays], ['T_{CPU}', 'T_{MTR}', 'ms'] )

Iteration 1 of 8

Using nudge: 0.002 

Connected to http://localhost:20000

About to run <__main__.TiltRecoveryLoop_Simple object at 0x7fb0442b70> : Tilt Insert Loop at 2021-07-12_15-20-31 with 4.0 Hz update frequency ...
Run log stored in output/SHAFT-TLT-EVENT-LOG_2021-07-12_15-20-31.txt
Running ...

About the start the thread
Thread 547087274496 was asked to write output/SHAFT-TLT_2021-07-12_15-20-52.xml
Wrote output/SHAFT-TLT_2021-07-12_15-20-52.xml !

--------- Tick 100 ---------

Root node, Name: Tilt Insert Loop , Status: Status.RUNNING


Root node Tilt Insert Loop failed!

[-] Tilt Insert Loop [x]
    --> Record_Classify [o]
    [-] Sequence [x]
        -^- Run_to_X_Failures_DECO [x] -- Allowed 1 of 1
            [o] Selector [x]
                -^- SuccessIsFailure [x] -- success is failure
                    [-] Sequence [o]
                        --> EventLogger [o]
                        [-] Jog_Safe [o]
                            --> Move_Arm [o]
                      

# Spiral Insert Process

In [6]:
from assembly_trees import _DUMMYPOSE
import py_trees

wheelPreInsertPose = wheelPostInsertPose.copy()
zApproach = 0.050
wheelPreInsertPose[2,3] += zApproach
print( "A pre-insert pose was defined." )


def get_spiral_test_tree():
    rootNode = Sequence_Recorder(  memory = 0 , dataKey = "test_DATA" , flagKey = "test_FLAG" , 
                                   outFileGenFunc = get_outfile_namer( pathPrefix = "output/" , 
                                                                           namePrefix = "GEAR-SPR_" )  )

    recordBehav = Record_Classify( recordFlagKey = "test_FLAG" , 
                                   dataKey       = "test_DATA" , 
                                   recordHz = 50, ctrl = robot )


    insertTree = Sequence( memory = 1 )
    
    waitBehav = Timeout_Success( timeout = 0.20 )

    goBehav = Jog_Safe( wheelGraspPose , 
                        zSAFE = 0.100 ,  
                        hover = 1 ,  
                        ctrl  = robot  )

    graspBehav = Grasp_at_Pose( wheelGraspPose, 0.026 ,
                                zApproach=0.050 , zClose=0.0 , wdthNarrow = 0.050 , maxIter=5, zFree=0.100 , 
                                ctrl = robot )

    jogBehav = Jog_Safe( wheelPreInsertPose, 
                         zSAFE = 0.100 ,  
                         hover = 0 ,  
                         ctrl  = robot  )

    relaxedInsert = Selector( name = "Spiral_ins_ALWAYSDROP" , memory = 1 )
    
    stepFactor = 2.0
    
    # 1.8 # Lateral torque caused it to bind
    insertBehav = Spiral_Insert( 
        wheelPreInsertPose ,
        touch_force = 2.0 , drop_force = 1.0 , insert_force = 9.0 , max_movement = zApproach + 0.030 , 
        lateralStopTorque = 0.475 , 
        pushbackF = 20.0 , spiralSpeed = 0.001 , descendSpeed = 0.00625 , biasWrist = 1 ,
        degrees_to_step = 3.25*2.0 , 
        maxAngle = 100 * 360 , startRadius = 0.00005 , 
        stepSize = 0.000025*1.75 , 
        max_radius = 0.010 ,
        posnMargin = 0.003 , orntMargin = 2.0/180 , 
        reliefStep_m = 0.0005 , reliefMargin_Nm = 0.050 , reliefN = 15 , 
        finalInsertHandZ = wheelPostInsertZ , 
        suppressDrop = 1 , suppressLateral = 0 , 
        recordFlagKey = "test_FLAG" , 
        ctrl = robot,
        chaseMode = 1, chaseStep = 0.00010 , N_chaseMax = 30 
    )                    
    
    dropBehav = Set_Fingers( openState = 0.060 , ctrl = robot ) 
    
    relaxedInsert.add_children( [ insertBehav , py_trees.decorators.SuccessIsFailure( dropBehav ) ] )
    
    cntrBehav = Jog_Safe( wheelPreInsertPose , #wheelPreOffsetYPose,  # wheelPreInsertPose , 
                         zSAFE = 0.100 ,  
                         hover = 0 ,  
                         ctrl  = robot  )
    
    insertTree.add_children( [ goBehav , graspBehav , jogBehav , relaxedInsert, dropBehav ] ) #, cntrBehav , checkBehav ] )

    rootNode.add_children( [ recordBehav , insertTree ] ) #, waitBehav ] )
    
    return rootNode

print( "Spiral BT Created!" )

A pre-insert pose was defined.
Spiral BT Created!


# Spiral Insert: Recording Pipeline

In [7]:
N        =  1
divSleep = 10
t_sleep  = 60
j        =  1

from time import sleep

for i in range( 1 , N+1 ):
    
    try:
        
        print( "Iteration" , i , "of" , N )
        
        # 6. Sleep for human reset
        if i > 1:
            sleep( divSleep )

        # 3. Release
        robot.hand.release()
        
        # 1. Reset tree
        rootNode = get_spiral_test_tree()
        
        # 2. Run tree and generate recording
        run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , 
                           breakOnFailure = 1 , breakOnSuccess = 1 , Nverb = 0 )
        
        # 3. Release
        robot.hand.set_finger_width( 0.050 ) 
        
        # 4. Deactivate motors
        robot.hand.deactivate_motors()
        
        # 5. Move to safe position
        robot.arm.move( safeCenterPose )

    except KeyboardInterrupt:
        print( "Pipeline was STOPPED by user at iteration" , i+1 )
        break
        

Iteration 1 of 1


NameError: name 'Sequence_Recorder' is not defined

# Common Functions

In [14]:
robot.arm.align_tcp( robot.arm )

True

In [19]:
robot.hand.grip()

True

In [ ]:
robot.hand.release()

In [18]:
robot.hand.set_finger_width( 0.040 ) 

True

In [ ]:
robot.arm.move( wheelGraspPose )

In [ ]:
print( robot.arm.get_tcp_pose() )
print( robot.arm.get_tcp_pose()[2,3] )
print( baseTouchPoseZ )

In [ ]:
robot.ft.get_wrist_force()

In [ ]:
robot.hand.deactivate_motors()

In [ ]:
robot.hand.zero_fingers()

# Setup

## Behavior: Move to Contact

In [15]:
rootNode =  Move_to_Contact( Fmag = 2.5 , relMove = [0.000, 0.000, -0.100] , biasWrist = 1 , 
                         mode = 'l' , speed = 0.0625 , accel = 0.17 , ctrl = robot )
print( "FT before the behavior:" , robot.ft.get_wrist_force() )
run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 )
print( "FT After the behavior:" , robot.ft.get_wrist_force() )

FT before the behavior: [-0.7564454078674316, 4.411328315734863, -1.97247314453125, -0.35438072681427, -0.0849374532699585, -0.12160936743021011]
Root node Move_to_Contact succeeded!


Simulation completed! with status rootNode.status 


FT After the behavior: [0.8244140148162842, -0.09648436307907104, -13.484954833984375, 0.025869131088256836, 0.23348236083984375, -0.008314445614814758]


## Behavior: Relieve Wrist Torque

In [18]:
rootNode =  Minimize_Wrist_Torque_XY( numTrials = 30 , marginT=0.05 , moveStep=0.0005  , ctrl = robot )
print( "FT before the behavior:" , robot.ft.get_wrist_force() )
run_BT_until_done( rootNode , N = 1000 , tickPause = 0.25 , breakOnFailure = 1 , breakOnSuccess = 1 )
print( "FT After the behavior:" , robot.ft.get_wrist_force() )


FT before the behavior: [0.22441411018371582, -0.2964843511581421, -11.6849365234375, 0.04186904430389404, 0.12548232078552246, -0.005314454436302185]
Root node Minimize_Wrist_Torque_XY succeeded!


Simulation completed! with status rootNode.status 


FT After the behavior: [-1.375585913658142, -0.396484375, -11.6849365234375, 0.04386913776397705, -0.11851763725280762, -0.00931444764137268]


## Setup Actions

In [7]:
robot.arm.move( shaftPreInsertPose )

In [8]:
robot.hand.set_finger_width( 0.030 ) 

True

In [9]:
robot.hand.grip()

True

In [6]:
robot.hand.release()

False

In [7]:
robot.arm.align_tcp()

True

### Gear Poses

In [13]:
# wheelGraspPose = robot.arm.get_tcp_pose()
# %store wheelGraspPose

Stored 'wheelGraspPose' (ndarray)


In [19]:
# wheelPostInsertPose = robot.arm.get_tcp_pose()
# wheelPostInsertZ = wheelPostInsertPose[2,3]
# %store wheelPostInsertPose
# %store wheelPostInsertZ
# wheelPreInsertPose = wheelPostInsertPose.copy()
# zApproach = 0.070
# wheelPreInsertPose[2,3] += zApproach
# %store wheelPreInsertPose

Stored 'wheelPostInsertPose' (ndarray)
Stored 'wheelPostInsertZ' (float64)
Stored 'wheelPreInsertPose' (ndarray)


In [ ]:
# wheelPreOffsetYPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetYPose

In [ ]:
# wheelPreOffsetXPose = robot.arm.get_tcp_pose()
# %store wheelPreOffsetXPose

### Shaft Poses

In [10]:
# shaftGraspPose = robot.arm.get_tcp_pose()
# %store shaftGraspPose

Stored 'shaftGraspPose' (ndarray)


In [11]:
# shaftPreInsertPose = robot.arm.get_tcp_pose()
# %store shaftPreInsertPose

Stored 'shaftPreInsertPose' (ndarray)


### General Poses

In [6]:
# safeCenterPose = robot.arm.get_tcp_pose()
# %store safeCenterPose

Stored 'safeCenterPose' (ndarray)


In [27]:
# baseTouchPose  = robot.arm.get_tcp_pose()
# baseTouchPoseZ = baseTouchPose[2,3]
# %store baseTouchPose
# %store baseTouchPoseZ

Stored 'baseTouchPose' (ndarray)
Stored 'baseTouchPoseZ' (float64)
